In [ ]:
import datetime
import pandas as pd
import requests


In [ ]:
def year_transfer(t):
    try:
        date = str(int(t[:t.index("/")])+1911)+t[t.index("/"):].replace('/','-')
    except:
        date=None
    return date

    

def crawl_company_basic_info_tw():
    data=[]
    market_category=["sii","otc","rotc"]
    for market in market_category:
        url = "https://mops.twse.com.tw/mops/web/ajax_t51sb01"
        form_data = {
            "encodeURIComponent": "1",
            "step": "1",
            "firstin": "1",
            "TYPEK": market
        }

        res = requests.post(url,data = form_data)
        res.encoding = "utf-8"
        df=pd.read_html(res.text)
        df=pd.DataFrame(df[0])
        data.append(df)

    df2=pd.concat(data)
    df2 = df2.astype(str)
    df2 = df2.apply(lambda s: s.str.replace(",", ""))
    df3=df2.loc[:,["公司代號","update_time","公司名稱","公司簡稱","產業類別","外國企業註冊地國","住址",
                   "董事長","總經理","發言人","發言人職稱","總機電話",
                   "成立日期","上市日期","上櫃日期","興櫃日期","實收資本額(元)","已發行普通股數或TDR原發行股數",
                   "私募普通股(股)","特別股(股)","普通股盈餘分派或虧損撥補頻率","股票過戶機構","簽證會計師事務所",
                   "公司網址","投資人關係聯絡電話","投資人關係聯絡電子郵件","英文簡稱"]]


    df3 = df3.rename(columns={
                                "公司代號":"stock_id","公司名稱":"name",
                                "公司簡稱":"short_name","產業類別":"category",
                                "外國企業註冊地國":"registered_country","住址":"address",
                                "董事長":"chairman","總經理":"ceo",
                                "發言人":"spokesman","發言人職稱":"spokesman_title",
                                "總機電話":"phone","成立日期":"establishment_date",
                                "上市日期":"sii_date","上櫃日期":"otc_date",
                                "興櫃日期":"rotc_date","已發行普通股數或TDR原發行股數":"shares_issued",
                                "私募普通股(股)":"private_shares","特別股(股)":"special_shares",
                                "普通股盈餘分派或虧損撥補頻率":"dividend_frequency","股票過戶機構":"stock_transfer_institution",
                                "簽證會計師事務所":"visa_accounting_firm","公司網址":"website",
                                "投資人關係聯絡電話":"investor_relations_contact","投資人關係聯絡電子郵件":"investor_relations_email",
                                "英文簡稱":"english_abbreviation","實收資本額(元)":"capital"

                                })
    #Data format Process
    df3=df3[df3["stock_id"]!="公司代號"]
    df3["registered_country"]=df3["registered_country"].apply(lambda s: s.replace("－", "台灣"))

    for share_column in ["capital","shares_issued","private_shares","special_shares"]:
        df3[share_column]=df3[share_column].apply(lambda s:pd.to_numeric(s, errors="coerce"))
        
    for date_column in ["establishment_date","sii_date","otc_date","rotc_date"]:
        df3[date_column]=df3[date_column].apply(lambda t:year_transfer(t))

    df3["update_time"]=datetime.datetime.now()
    df3 = df3.fillna('')
    return df3

In [ ]:
df=crawl_company_basic_info_tw()
df

In [ ]:
#pycharm 測試
import sys,os
sys.path.append("..")
import django
django.setup()
from crawlers.models import *
# from crawlers.finlab.pioneers import *
# from crawlers.finlab.import_tools import *

In [ ]:
add_to_sql(CompanyBasicInfoTW, df)